In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import seaborn as sns
from datetime import datetime, timedelta
from scipy.stats import pearsonr
from scipy.stats import kendalltau
from scipy.stats import spearmanr
import os
from scipy.stats import binom
from scipy.special import comb

In [2]:
## Definition der Transitionmatrix
def get_Q(i, t, alpha, gamma, EM_it_1, Y_it, O_it, NS):
    X_it_1 = [EM_it_1, EM_it_1**2, int(Y_it > 0), int(O_it > 0)]
    
    # v_(it,s→s')
    v = np.zeros((NS, NS))
    for s in range(NS):
        for s_prime in range(NS):
            v[s, s_prime] = alpha[s, s_prime] + gamma[s, s_prime] * X_it_1
    
    # q_(it, s->s')
    q = np.zeros((NS, NS))
    for s in range(NS):
        denominator = 1 + np.sum(np.exp(v[s, 1:]))
        for s_prime in range(NS):
            if s_prime == 0:
                q[s, s_prime] = 1 / denominator
            else:
                q[s, s_prime] = np.exp(v[s, s_prime]) / denominator
    
    return q

In [ ]:
def emission_prob_o(o_it, s, EM_it, O_it, alpha_os, beta_io, sigma_alpha_o, sigma_beta_o):
    # Compute the parameter p_it_s
    LO_it = O_it[s][EM_it]
    alpha_io = alpha_os.reshape(1, -1) + np.random.normal(scale=sigma_alpha_o, size=(NS, 1))
    beta_io = beta_io + np.random.normal(scale=sigma_beta_o, size=NS)
    p_it_s = np.exp(alpha_io + beta_io * np.log(LO_it)) / (1 + np.exp(alpha_io + beta_io * np.log(LO_it)))
    
    # Compute the probability of the observed value
    prob = 0
    for i in range(EM_it + 1):
        prob += comb(EM_it, i) * (p_it_s ** i) * ((1 - p_it_s) ** (EM_it - i)) * (o_it ** i) * ((1 - o_it) ** (EM_it - i))
    
    return prob


In [ ]:
def P_O_it_given_s(o_it, s, EM_it, LO_it, alpha_os, beta_io, sigma_alpha_o, sigma_beta_o):
    # Berechne die Wahrscheinlichkeit p_it_s
    p_it_s = 1 / (1 + np.exp(-(alpha_os[s] + beta_io * np.log(LO_it))))
    
    # Berechne die Anzahl der Öffnungen
    k = o_it
    
    # Berechne die Wahrscheinlichkeit P(O_it = o_it | s)
    p_O_it_given_s = binom.pmf(k=k, n=EM_it, p=p_it_s)
    
    return p_O_it_given_s


In [ ]:
def P_O_it_given_s(o_it, s, EM_it, LO_it, alpha_o_s, beta_io, sigma_alpha_o, sigma_beta_o):
    p_it_given_s = 1 / (1 + np.exp(-(alpha_o_s[s] + beta_io * np.log(LO_it))))
    p = binom.pmf(o_it, EM_it, p_it_given_s)
    alpha_io = alpha_o_s[s] + np.random.normal(scale=sigma_alpha_o, size=NS)
    beta_io = beta_io + np.random.normal(scale=sigma_beta_o, size=NS)
    return p, alpha_io, beta_io


In [ ]:
def P_O_it_given_s(LO_it, o_it, s, EM_it, alpha_o, beta_io, sigma_alpha_o, sigma_beta_o):
    """
    Berechnet die Wahrscheinlichkeit P(O_it=o_it | s) für gegebenen Zustand s und
    Beobachtung LO_it=o_it unter Verwendung der Binomialverteilung.

    Args:
        LO_it (float): Beobachtung von Variable LO zur Zeit t im Zustand i
        o_it (int): Beobachtung von Variable O zur Zeit t im Zustand i
        s (int): Zustand s
        EM_it (int): Anzahl der Versuche (Emissionen) zur Zeit t im Zustand i
        alpha_o (array): Array mit den Alpha-Parametern für jeden Zustand
        beta_io (float): Beta-Parameter für Zustand i und Variable O
        sigma_alpha_o (float): Standardabweichung von Alpha
        sigma_beta_o (float): Standardabweichung von Beta

    Returns:
        float: Wahrscheinlichkeit P(O_it=o_it | s)
    """
    p_it_given_s = np.exp(alpha_o[s] + beta_io * np.log(LO_it)) / (1 + np.exp(alpha_o[s] + beta_io * np.log(LO_it)))
    p_o_given_it = binom(EM_it, o_it) * np.power(p_it_given_s, o_it) * np.power(1 - p_it_given_s, EM_it - o_it)
    return p_o_given_it


In [ ]:
def P_O_it_given_s(LO_it, o_it, s, alpha_o, beta_o, alpha_io, beta_io, sigma_alpha_o, sigma_beta_o, EM_it):
    p_it_given_s = 1 / (1 + np.exp(-(alpha_io + beta_io * np.log(LO_it))))
    likelihoods = binom.pmf(o_it, EM_it, p_it_given_s)
    return likelihoods.prod(axis=1)


In [ ]:
import numpy as np
from scipy.optimize import minimize

# Define the HMM model with a binomial emission distribution
from hmmlearn import hmm

model = hmm.MultinomialHMM(n_components=2, n_iter=100, tol=0.01, verbose=True)

# Generate simulated data
X = np.random.randint(0, 2, size=(100, 10))
lengths = np.full(100, 10)

# Define the negative log-likelihood function
def neg_log_likelihood(params):
    alpha, beta = params[:2], params[2:]
    model.startprob_ = alpha / np.sum(alpha)
    model.transmat_ = np.array([[beta[0], 1 - beta[0]], [1 - beta[1], beta[1]]])
    model.emissionprob_ = np.array([[[1 - p, p], [p, 1 - p]] for p in params[4:]])
    return -model.score(X, lengths)

# Estimate the parameters using maximum likelihood
init_params = np.array([1, 1, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])
result = minimize(neg_log_likelihood, init_params, method='BFGS')
alpha, beta, p = result.x[:2], result.x[2:4], result.x[4:]
